In [9]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

cm.update('livereveal', {
              'width': "90%",
              'height': "90%",
              'scroll': True,
})

import warnings
warnings.filterwarnings('ignore')

bShowInline = True  # Set = False for document generation
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from ipywidgets import widgets, Layout, Box, GridspecLayout


##Basic mcq

def create_multipleChoice_widget(description, options, correct_answer, hint):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            s = '\x1b[5;30;41m' + "try again" + '\x1b[0m' +"\n"
        with feedback_out:
            feedback_out.clear_output()
            print(s)
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    hint_out = widgets.Output()
    
    def hint_selection(b):
        with hint_out:
            print(hint)
            
        with feedback_out:
            feedback_out.clear_output()
            print(hint)
    
    hintbutton = widgets.Button(description="hint")
    hintbutton.on_click(hint_selection)
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         widgets.HBox([hintbutton, check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 

test = create_multipleChoice_widget('1. Let a, b, c be real numbers. If a<b and b<c, then:',
                                   ['a>c','a<c','b<c','c<a'],
                                   'a<c',
                                   "[hint]:")

pingpong = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'D',"[hint]: assume the surface is static over time")

ambi = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'B',"[hint]: check the units!")

sens = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'C',"[hint]: set b = 0 in code cell and recalculate")

los = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'A',"[hint]: most spaceborne radars point closer to nadir than the horizon to maximize signal return")

berr = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'C',"[hint]: is the line-of-sight vector a constant across the swath?")

<img src="Figs/Banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="6"> <b> Synthetic Aperture Radar Interferometry <b> </font>

<font size="4"> <b> Geometric Aspects of InSAR <font color='rgba(200,0,0,0.2)'> </font> </b> </font>

<br> <img src="Figs/NASALogo.png" width="250" align="right" /> <br> 
<font size="4"> <b> Paul A Rosen and Scott Hensley</b> 
<font size="3">  <br>
<font> <b>Date: </b> July 31 , 2021 </font>
</font>


This notebook presents the basics of synthetic aperture radar interferometry. The student will be exposed to geometric aspects of interferometry.  The key concepts are:

1. The relationship of the phase of an radar wave to the propagation distance
2. The concept of wave interference depending on the relative phase of two waves converging at a point
3. The relationship of the relative phase of two observations taken from different vantage points to topography
4. The relationship of the relative phase of two observations taken at different times to surface displacement
5. The differences in sensitivity of interferometry for topography and for surface displacement



# Review the SAR Notebook

There are equations, variables, and concepts in this notebook that rely on some knowledge of SAR, as described in the SAR notebook.  Do you know what $s(\rho,x)$ is?  Backscatter? Range? Azimuth? Layover? Shadow? Speckle? Resolution? If not, please review the SAR notebook before exploring here! 

# Interferometry

## The relationship of phase to propagation distance

The phase of a propagating wave is directly proportional to the distance propagated.  

\begin{equation}
\phi = 
\underbrace{\quad 2\pi \quad}_\text{Radians per wavelength} \cdot \underbrace{\quad  \frac{\rho}{\lambda} \quad}_\text{Number of wavelengths}
\end{equation}

where $\rho$ is the distance along the propagation path, $\lambda$ is the wavelength of the wave, and $\phi$ is the phase of the wave.  <img src="Figs/phsdist.png" width="350" align="right" />

It is difficult to determine the absolute phase at a point along the propagation path, but the relative phase - the difference in phase between two waves measured at a point - can be measured more easily.

Radar interferometry can be broadly defined as the use of phase measurements to determine the relative distance to an object when imaged by synthetic aperture radar from two or more observations separated either in time or space. Interferometric phase is simply another means of measuring relative distance.


Phase measurements in interferometric systems can be made with degree-level accuracy.  With typical radar wavelengths in the 3-80 cm range, this corresponds to relative range measurements having mm accuracy.



## Wave Interference

The phase difference of the two waves at a point is proportional to the relative propagation distance of each wave to that point.  <img src="Figs/waveint.png" width="500" align="right" /> 

The phase difference observed along a surface creates an interference pattern, which captures the different paths the waves took to reach each point. The wave amplitude is varying sinusoidally in time, but the relative phase at each point is constant.  If the observing surface could capture phase differences, each point would have a fixed value between $[ -\pi,\pi]$.

## SAR Interferometry

A SAR image is created by a radar system transmitting a wave and receiving its echo from the ground as the radar flies along its path. <img src="Figs/int_topo.png" width="500" align="right" />

Referring to the figure, the wave from a SAR then illuminates the surface, and the phase of the wave at each point on the surface will differ depending on the distance from the radar to the surface.  

With a second radar observing from a different vantage point, the relative phase between the two observations then encodes the relative distance to a point on the surface, which in this case varies with the topographic relief. 


## SAR Interferogram

The image on the left is a standard SAR image - grey scale representing the radar reflectivity of the surface, including the intrinsic radar cross section and area projection differences caused by the topographic relief.  <img src="Figs/sarintimg.png" width="500" align="right" /> 

The phase (not shown) would appear random from pixel to pixel in the image due to the coherent backscatter effects known as speckle (see the SAR module).

The image on the right is an "interferogram", which is the constructed by forming the complex cross-product of the one image with a second acquired from a slightly different vantage point as illustrated on the previous slide. 
\begin{eqnarray}
i(\rho,x) & = & s_1(\rho,x) \cdot s_2^*(\rho,x)\\
& = & | s_1 | | s_2 | e^{j(\phi_1-\phi_2)}
\end{eqnarray}
where $\rho$ is the range, $x$ is the position along the flight direction (into the computer screen), and $s_1$ and $s_2$ are the two complex images.  One cycle of color represents $2\pi$ of relative phase. 

The interferogram is a complex image.  Its magnitude is related to correlation, and its phase to observation geometry differences.

# Interferometry for Topography

## The interferometric triangle

Referring to the figure to the right, the measured phase difference in the interferogram is <img src="Figs/topotriang.png" width="400" align="right" />  

\begin{equation}
\Delta\phi = \phi_1-\phi_2 = \frac{2\pi p}{\lambda} \delta\rho
\end{equation}
where $p$ is an integer value of 1 or 2, depending on how the interferogram was acquired (explained on the next slide).

Having measured $\Delta \phi$, we can solve for $\delta\rho$.  With $\delta\rho$, we can use the law of sines to triangulate the position on the ground:

\begin{eqnarray}
\sin(\theta-\alpha) & = & \frac{(\rho+\delta\rho)^2-\rho^2-B^2}{2\rho B} \\
z & = & h=\rho\cos\theta
\end{eqnarray}

To triangulate as above, the $\rho$, $B$, and $\alpha$ must be know to sufficient accuracy to achieve the desired topographic accuracy.  Typically, $B$ must be known to mm, $\alpha$ to the angular equivalent, and $\rho$ to meters.  The system must also be phase stable to mm equivalents.

## Data Collection Method defines $p$

For single-pass interferometry, two radar antennas are present at the same time and form the interferometer.  These can be mounted on the same platform as depicted in the figure, or they could be co-flying on separate platforms.  In either case, there are two modes of operation that affect the interferometric phase. <img src="Figs/pingpong.png" width="550" align="right" />  

1. "Classic" single-antenna transmit mode: one antenna transmits and both receive. $p=1$
2. "Ping pong" dual-antenna  mode: each antenna transmits and receives its own echo, effectively doubling the physical baseline. $p=2$


<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Interferometer Methods <b> </font>
</div>

What would be the value of $p$ for an interferometer formed by a single radar system orbiting precisely enough to nearly repeat its track, such that the interferogram is formed by comparing the first observation to the second (repeat-pass interferometry)?

A. $p=3$

B. $p=0$

C. $p=1$

D. $p=2$

In [11]:
pingpong

## Relating the interferometric phase to the interferometric baseline

The interferometric phase can be approximated as

\begin{equation}
\Delta\phi \approx - \frac{2 \pi p}{\lambda} \langle \hat{\ell},\vec{b}\rangle
\end{equation}
where $\langle \, ,\rangle$ denotes the dot product.

With $\vec{b} = (b \cos\alpha,b\sin\alpha)$ and $ \hat{\ell} = (\sin\theta,-\cos\theta) $,  then

\begin{equation}
\Delta\phi \approx - \frac{2 \pi p}{\lambda}b \sin( \theta-\alpha)
\end{equation}

<img src="Figs/bpar_phs.png" width="300" align="left" /> Referring to the figure to the left, we see that the interferometric phase is proportional to the projection of the baseline vector $\vec{b}$ on the unit look vector.  This is referred to often as the "parallel component of the baseline"

## Phase Gradient

* For a number of applications including flight planning and unwrapping studies it is desirable to be able to compute the interferometric phase gradient for an arbitrarily sloped surface, look geometry and baseline.

* The interferometric phase is well approximated for most applications  by
\begin{equation*}
\Delta\phi = -\frac{2\pi p}{\lambda} \langle \vec{b}, \hat{\ell} \rangle
\end{equation*}
where  $\vec{b}$ is the baseline vector,  $\hat{\ell}$ a unit vector pointing to the target and $\lambda$ is the wavelength, and $\langle \, , \rangle$ is the inner, or dot, product.

* The phase gradient is
\begin{equation*}
\nabla\Delta\phi =  \begin{bmatrix}\frac{\partial \Delta\phi}{\partial s} \\ \frac{\partial\Delta\phi}{\partial \rho} \end{bmatrix}
=  -\frac{2\pi p}{\lambda} \begin{bmatrix}\langle \vec{b}, \frac{\partial\hat{\ell}}{\partial s} \rangle \\ \langle \vec{b}, \frac{\partial\hat{\ell}}{\partial \rho} \rangle \end{bmatrix}
\end{equation*}
where $s$ is the along track coordinate and $\rho$ is the range.

## Phase Gradient Observations

* There in a change in phase with respect to range regardless of whether the terrain is sloped in the range direction or not. The phase rate, or fringe rate, with respect to a flat surface is called the flat surface (or spherical earth) fringe frequency. 

\begin{equation*}
\frac{\partial\Delta\phi}{\partial \rho} = \frac{2\pi p}{\lambda} \frac{b \cos(\theta - \alpha)}{\rho\tan(\theta -\psi_c)} =\frac{2\pi p}{\lambda} \frac{b_\perp}{\rho\tan(\theta -\psi_c)}
\end{equation*}

* Note that the fringe rate depends on the local slope and the perpendicular baseline length.

* The fringe rate in the azimuth or along track direction is zero unless there is an azimuth slope. It also depends on the magnitude of the local slope and the perpendicular baseline length

\begin{equation*}
\frac{\partial\Delta\phi}{\partial s} = \frac{2\pi p}{\lambda} \frac{b \cos(\theta - \alpha)}{\rho} \sin(\theta)\tan(\psi_s) =\frac{2\pi p}{\lambda} \frac{b_\perp}{\rho} \sin(\theta)\tan(\psi_s)
\end{equation*}


## Flattened Phase

In many interferometric workflows, either through explicit motion compensation to a common reference of images as they are formed, or at some later point in the interferogram formation,  the flat Earth fringes are removed, leaving an interferogram with fringes that are proportional to the topography.  These are referred to as "flattened phase" or "flattened fringes".

The flattened phase is given by 

\begin{equation}
\Delta\phi_{\rm flat} = - \frac {2 \pi p}{\lambda}\bigg( \langle\hat{\ell},\vec{b}\rangle - \langle\hat{\ell}_0,\vec{b}\rangle \bigg)
\end{equation}

where $\hat{\ell}$ is the look vector to a point and $\hat{\ell}_0$ is the corresponding look vector to the flat surface at the same range.

Making the usual 2-dimensional simplification, 

\begin{eqnarray}
\Delta\phi_{\rm flat} & = & - \frac {2 \pi p}{\lambda} b \big(\sin(\theta-\alpha) - \sin(\theta_0-\alpha) \big) \\
 & \approx & - \frac {2 \pi p}{\lambda} b \cos(\theta-\alpha) \delta\theta \\
 & \approx & - \frac {2 \pi p}{\lambda} b_{\perp} \delta\theta \\
 \end{eqnarray}
 
<img src="Figs/intgeom.png" width="350" align="left" /> From the figure we have 
\begin{equation}
\delta\theta \approx \frac{h_T}{\rho\sin\theta_0}
\end{equation}

which gives 

\begin{eqnarray}
\Delta\phi_{\rm flat}(\rho,x) & \approx & - \frac {2 \pi p}{\lambda} b_{\perp} \delta\theta \\
                  & \approx & - \frac {2 \pi p}{\lambda} b_{\perp} \frac{h_T(\rho,x)}{\rho\sin\theta_0} \\
\end{eqnarray}

where $x$ is the position in the flight direction in the image, and the dependence on position of the height, and consequently the phase is made explicit.

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Ambiguity Height <b> </font>
</div>

The ambiguity height is defined as the change in elevation required to change the interferometric phase by $2 \pi$ radians.  Which of the following formulas would define the "ambiguity height"?

A. $h_{\rm amb} = \frac{p b_\perp}{\lambda\rho\sin\theta}$

B. $h_{\rm amb} = \frac{\lambda\rho\sin\theta}{p b_\perp}$

C. $h_{\rm amb} = \frac{\lambda p b_\perp}{\rho\sin\theta}$

D. $h_{\rm amb} = \frac{2 \pi \lambda\rho\sin\theta}{p b_\perp}$

In [3]:
ambi

# Differential Interferometry


## Repeat-pass interferometry 

When two observations are made from the same position in space but at different times, the interferometric phase is proportional to any change in the range of a surface feature directly, often referred to the displacement of the surface in the range direction. <img src="Figs/diffint.png" width="550" align="left" /> 

The phase difference is proportional to the range difference:

\begin{eqnarray}
\Delta\phi & = & - \frac{4\pi}{\lambda} \bigl(\rho(t_1) -\rho(t_2)\bigr)\\
& = & \frac{4\pi}{\lambda} \Delta\rho_{\rm change}\\
 & \approx &  \frac{4\pi}{\lambda} \langle \vec{D}, \hat{\ell} \rangle
\end{eqnarray}
 
 where $\vec{D}$ is the vector displacement of the surface from one time to the next, and the range displacement is characterized as the projection of  $\vec{D}$ on the look direction $\vec{\ell}$.

## Deformation Phase Convention

The relationship between phase and surface deformation depends on the sign convention employed. The usual interferogram sign convention has the conjugate applied to the data that is most recent in time. So in the above, $t_2 > t_1$ and since the range-phase relationship for an image is $\phi = - 4\pi\rho/\lambda$,

\begin{equation}
\Delta\phi = \arg(s(t_1) s^*(t_2)) = - \frac{4\pi}{\lambda} \bigl(\rho(t_1) -\rho(t_2)\bigr) = \frac{4\pi}{\lambda} \bigl(\rho(t_2) -\rho(t_1)\bigr)
\end{equation}

This is illustrated in the figures below.  On the left, range/phase decrease in time, which would be caused by motion toward the radar along the line of sight, while on the right, range/phase increase in time, indicating motion away from the radar along the line of sight. <img src="Figs/relmotion.png" width="900" align="center" />



## Differential interferometry and topography

In general, it is difficult to repeat the observation position exactly, so the phase difference will have a component of the topography also: <img src="Figs/diffint_topo.png" width="350" align="left" />

\begin{equation}
\Delta\phi = \frac{4\pi}{\lambda} \bigl[ \underbrace{ - \langle \vec{b}, \hat{\ell}\rangle}_\text{Topography Term} +  \underbrace{\langle \vec{D}, \hat{\ell} \rangle}_\text{Displacement Term}\bigr]
\end{equation}

This interferogram can be flattened as described above, and is described

\begin{equation}
\Delta\phi_{\rm flat}(\rho,x) = \frac {4\pi}{\lambda} \Delta\rho_{\rm change}(\rho,x) - \frac {2 \pi p}{\lambda} b_{\perp} \frac{h_T(\rho,x)}{\rho\sin\theta_0}
\end{equation}

where the dependence of the range change and the topography on the image position $(\rho,x)$ is made explicit.  

Note that a small change in range (cm) over time can have a similar magnitude to a meter scale change in topography.  Without further information, it is difficult to separate the two.  If topography is known, then the topography term can be estimated and removed to reveal the surface displacement.


<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Line of Sight Displacement <b> </font>
</div>

From the above description, it should be clear that while a displacement over time will in general be a vector in three-dimensional space, in a single inteferogram it is only possible to measure the displacement that has a component in the line of sight direction: $\Delta\rho=\langle \vec{D}, \hat{\ell} \rangle$  Using the figure above that illustrates a simple right-looking geometry as a guide, for a polar-orbiting satellite looking right, perpendicular to its track, at an incidence angle of 30 degrees, which components of displacement would an interferogram be most sensitive to?

A. east (i.e. right) and up

B. north (i.e. into the page)

C. east 

D. north and east

In [4]:
los

##  Interferometry Sensitivities

* Differential interferometry can detect millimeter level surface deformation

* Differential phase is much more sensitive to displacements than to topography

$\qquad \qquad{\rm Topographic \ Sensitivity:} \frac{\partial\Delta\phi}{\partial h} = \frac{2\pi p b \cos(\theta-\alpha)}{\lambda \rho \sin \theta}$

$\qquad \qquad{\rm Displacement \ Sensitivity:} \frac{\partial\Delta\phi}{\partial \Delta\rho} = \frac{4\pi}{\lambda}$

* Since $b/\rho \ll 1$, it takes a much larger change in $h$ than in $\Delta\rho$ to affect a change in $\Delta\phi$

* "Meter-scale topography - millimeter scale topographic change"

In [18]:
import numpy as np

theta = 30. * np.pi / 180.
p = 2
b = 100.
alpha = 0.
rho = 1000000.
wavl = 0.24

d_Delta_phi_d_h = 2.*np.pi * p * b * np.cos(theta-alpha)/(wavl * rho * np.sin(theta))
d_Delta_phi_d_Delta_rho = 4.*np.pi /wavl

print ("Sensitivity of phase to a change in topography based on triangulation is \n\n          ",d_Delta_phi_d_h," in radians per meter of elevation change\n")
print ("Sensitivity of phase to a line of sight displacement based on differential interferometry is \n\n          ", d_Delta_phi_d_Delta_rho, " in radians per meter of range change")

Sensitivity of phase to a change in topography based on triangulation is 

           0.00906899682117109  in radians per meter of elevation change

Sensitivity of phase to a line of sight displacement based on differential interferometry is 

           52.35987755982989  in radians per meter of range change


<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Sensitivity <b> </font>
</div>

What happens to the sensitivities when the baseline goes to zero?

A. Nothing changes

B. Displacement sensitivity great reduces

C. Topographic sensitivity vanishes

D. Topographic sensitivity vanishes and Displacement sensitivity greatly increases

In [5]:
sens

# Phase Unwrapping and Error Sources

## Phase unwrapping

<img src="Figs/unwrapping1.png" width="300" align="left" />The interferometric phase difference for a general interferogram is given by

\begin{equation}
\Delta\phi = \frac{4\pi}{\lambda} \bigl[ - \langle \vec{b}, \hat{\ell}\rangle + \langle \vec{D}, \hat{\ell} \rangle\bigr]
\end{equation}
however this phase cannot be measured directly.   Since the interferogram is a complex image, the phase in each pixel can only be calculated modulo $2\pi$

\begin{equation}
\Delta\phi_{\rm measured} = \tan^{-1}\frac{\Im(i)}{\Re(i)} 
\end{equation}
where $i=|i| \exp(j\Delta\phi)$ is the interferogram.  This implies that

\begin{equation}
\Delta\phi_{\rm measured} = \mod(\Delta\phi,2\pi) 
\end{equation}

<img src="Figs/unwrapping2.png" width="300" align="right" />

The unwrapped phase is calculated to reconstruct the phase that represents the physical range differences, either due to topography parallax, temporal range change, or both.  The example of one-dimensional phase unwrapping shown on the left is simple, though even in this case the solution is only known to within a constant.  

\begin{equation}
\Delta\phi_{\rm unwrapped}(\rho,x) = \Delta\phi(\rho,x) + {\rm constant}
\end{equation}

In general, in two dimensional noisy data, phase unwrapping can be quite challenging.

## Interferometric Error Sources

There are many error sources that we need to become familiar with.   Some of these are intrinsic to the technique: 

- Propagation delay errors from atmosphere, including the troposphere and ionosphere
- Phase unwrapping errors
- Interferometric decorrelation

In this course, we'll explore each of these in some more detail.

Others are related to the design of the imaging system itself.  These can often be controlled through system design, for example selection of look angles, antenna size, resolution, and overall calibration of the system:

- Layover and shadow in radar imagery from slant range geometry
- Multiple scattering within and among resolution cells
- Range and azimuth ambiguities due to design constraints
- Range and azimuth sidelobes due to bandwidth/resolution constraints
- Multipath and channel cross-talk noise as low-level interference
- Calibration errors
- DEM errors

In general, the errors in a displacement interferogram that has had its topography removed using a DEM will have the following residual errors in addition to the signal of interest:

\begin{equation}
\delta\Delta\rho = \frac{\lambda}{4\pi} \biggl[ \underbrace{\Delta\phi_{\rm int}}_\text{Phase Noise} +  \underbrace{\frac{4\pi b_\perp \delta h_{\rm DEM}}{\lambda\rho\sin\theta}}_\text{DEM Error} +\underbrace{\Delta\phi_{\rm atm}}_\text{Atmosphere Error} \biggr] + \underbrace{\langle \delta\vec{b}, \hat{\ell}\rangle}_\text{Baseline Calibration Error}
\end{equation}

where $\Delta\phi_{\rm int}$ represents the various pixel-to-pixel noise sources in the interferogram, due to thermal noise relative to the signal level, ambiguities and other multiplicative noise sources, interference; $\delta h$ is the error in knowledge ot the topographic height due to imperfections in the DEM -- for spaceborne sensors, this is typically linear or quadratic phase variation in range and/or azimuth; $\Delta\phi_{\rm atm}$ is the aggregate error due to atmospheric noise, including troposphere and ionosphere; and $\delta b$ is the error in the knowledge of the baseline.

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Baseline Error <b> </font>
</div>

Orbits are generally known to a few cm accuracy, varying slowly over the orbit.  What would be the nature of the phase residual for a 3 cm baseline error if the radar wavelength is 3 cm?

A. The interferometric phase residual would be on the order of $4\pi$ radians, but it would be a constant so does not effect relative displacement measurement  

B. The interferometric phase residual would be a function of the topography, varying at the level of $4\pi$ radians

C. The interferometric phase residual would be on the order of $4\pi$ radians, but it would be vary smoothly across range

D. The interferometric phase residual would negligible because 3 cm is much smaller than typical topographic variation

In [6]:
berr

## DEM Error Example

Phase errors arise in interferograms if the digital elevation model has height errors, and the perpendicular baseline is non-zero, which is generally true somewhere in the interferogram (why is that?). This was particularly problematic before 2000, when global digital topography was not uniformly accurate or available.  In 2000, NASA flew the Shuttle Radar Topography Mission (SRTM), which collected high-quality digital topography in the latitude range between $-60^\circ$ and $60^\circ$.  The figure below shows an interferogram of Mount Etna processed with two different DEMs.  The left figure used a pre-SRTM DEM, while the right figure used SRTM.  It is clear that the DEM can have a strong influence on the residuals.  The red arrow points to a particularly erroneous area, but it is clear that many of the residual fringes across this interferogram are due to the DEM, including in the area of maximum displacement near the center. <img src="Figs/demerr.png" width="500" align="center" />

## Tropospheric Noise

The troposphere has a variable refractive index in space and time, and a radar wave propagating through the troposphere will experience a spatially variable path delay, which translates directly to phase delay. The variations are primarily due to water vapor in the atmosphere.  The figure below illustrates a typical two-dimensional pattern that might be seen in an interferogram, even with no ground displacement. <img src="Figs/tropoerr2.png" width="600" align="center" />  

The interferometric phase when considering the troposphere is given by

\begin{equation}
\Delta\phi(\rho,x) = \frac{4\pi}{\lambda}\bigl( \delta\rho_{\rm disp}(\rho,x) + \delta\rho_{\rm atm} (\rho,x) \bigr) + {\rm noise}
\end{equation}

This equation has two unknowns $\delta\rho_{\rm disp}$ and $\delta\rho_{\rm atm}$ which are indistinguishable.  As a result this error source is particularly confounding because it is difficult to distinguish from the desired displacement $\delta\rho_{\rm disp}$, having similar spatial characteristics.  It is typically mitigated by looking at many different interferograms spanning a common event, since the atmosphere changes from observation to observation.  Persistent patterns in these interferograms might be the desired displacement (unless it is changing over time also!).

## Tropospheric Noise Example

As a particularly problematic example of troposphere-induced phase error, the left figure below shows phase signatures that are observed in an interfeogram over large volcanoes on the Big Island of Hawaii.  Since these volcanoes are active, one might conclude that they are deforming.  However, the center figure shows a calculation of expected interferometric phase induced by the difference in tropospheric path delay between the two observation times.  The signature is quite similar to the left figure.  The difference of these two is shown on the right - indicating that the majority of the observed phase is due to the troposphere.<img src="Figs/hawaii_tropo.png" width="800" align="center" />


<div class="alert alert-danger">
<font size="5"> <b> Homework <b> </font>
</div>
    
An interferometer flying over a flat Earth has the following characteristics:

- wavelength $\lambda=0.24$
- altitude $h= 600 {\rm km}$ 
- baseline length $|b| = 100 {\rm m}$
- baseline angle $\alpha = -60^\circ$
- look angle in the near range $\theta_{\rm nr} = 30^\circ$
- look angle in the far range $\theta_{\rm fr} = 45^\circ$
    
Calculate the ranges $\rho_{\rm nr}$ and $\rho_{\rm fr}$, and the interferometric phase rate (fringe rate) $\partial\Delta\phi/\partial\rho$ at each of these locations.  
    
   

# Other Resources: 


* Treatise on Geophysics, Interferometric Synthetic Aperture Radar Geodesy (2015), Simons M. and Rosen, P.; Schubert, G. (ed.), 2nd Ed., Volume 3- Geodesy, Elsevier Press, 339-385; http://web.gps.caltech.edu/~simons/publications/pdfs/Simons_etal_2015.pdf


* Synthetic Aperture Radar Interferometry (2000) Rosen, P. A.; Hensley, S.; Joughin, I. R.; Li, F. K.; Madsen, S. N.; Rodriquez, E. & Goldstein, R. M. Proc. IEEE, 88, 333-382


* Synthetic Aperture Radar Interferometry to measure Earth's surface topography and its deformation (2000) Burgmann, R.; Rosen, P. & Fielding, E. Ann. Rev. Earth Planet. Sci., 28, 169-209

* Synthetic Aperture Radar Interferometry: Principles and Applications (1999) Madsen, S. N. & Zebker, H. A., Manual of Remote Sensing, Artech House, 3


* Radar interferometry and its application to changes in the earth's surface (1998) Massonnet, D. & Feigl, K. L., Reviews of Geophysics, 36, 441-500


* Synthetic Aperture Radar Interferometry (1998) Bamler, R. & Hartl, P., Inverse Problems, 14, 1-54


* Remote Sensing with Imaging Radar (2009) Richards, J.A., Springer. 
